# Data loading - check to correctly understand data / label format

- load data from conformal boundary example dataset
- watch how data changes formats / shapes throughout the training pipeline.

In [2]:
import os

import pytorch3dunet
import pytorch3dunet.unet3d as u3  # !
from pytorch3dunet import predict
from pytorch3dunet import train

from pytorch3dunet.unet3d import trainer as trainer
from pytorch3dunet.unet3d.config import _load_config_yaml

from torchsummary import summary

In [3]:
sam = True  # (False if ben) thinking we could maintain different filepaths here -- or we could copy the whole notebook and maintain two different working notebooks.



In [16]:
config_dir = './3DUnet_confocal_boundary'
config_filename = 'test_config.yml'
os.listdir(config_dir)

['test_config.yml',
 'test_config.yml~',
 'ovules_raw.png',
 'ovules_pred.png',
 'train_config.yml']

In [17]:
config_filepath = f'{config_dir}/{config_filename}'
config = _load_config_yaml(config_filepath)
config

{'model_path': 'PATH_TO_BEST_CHECKPOINT',
 'model': {'name': 'UNet3D',
  'in_channels': 1,
  'out_channels': 1,
  'layer_order': 'gcr',
  'f_maps': 32,
  'num_groups': 8,
  'final_sigmoid': True},
 'predictor': {'name': 'StandardPredictor'},
 'loaders': {'output_dir': '/scratch/groups/jyeatman/samjohns-projects/unet3d/data/test-checkpoints',
  'batch_size': 1,
  'num_workers': 2,
  'test': {'file_paths': ['/scratch/groups/jyeatman/samjohns-projects/unet3d/data/osfstorage-archive-test'],
   'slice_builder': {'name': 'SliceBuilder',
    'patch_shape': [80, 170, 170],
    'stride_shape': [40, 90, 90]},
   'transformer': {'raw': [{'name': 'Standardize'},
     {'name': 'ToTensor', 'expand_dims': True}]}}}}

In [31]:
loaders = pytorch3dunet.datasets.utils.get_test_loaders(config)

In [19]:
for l in loaders:
    print(l)

2023-05-17 00:12:38,667 [MainThread] INFO Dataset - Creating test set loaders...
2023-05-17 00:12:38,668 [MainThread] WARNING Dataset - Cannot find dataset class in the config. Using default 'StandardHDF5Dataset'.
2023-05-17 00:12:38,669 [MainThread] INFO HDF5Dataset - Loading test set from: /scratch/groups/jyeatman/samjohns-projects/unet3d/data/osfstorage-archive-test/N_511_final_crop_ds2.h5...
2023-05-17 00:12:40,499 [MainThread] INFO Dataset - Slice builder config: {'name': 'SliceBuilder', 'patch_shape': [80, 170, 170], 'stride_shape': [40, 90, 90]}
2023-05-17 00:12:40,583 [MainThread] INFO HDF5Dataset - Number of patches: 432
2023-05-17 00:12:40,584 [MainThread] INFO HDF5Dataset - Loading test set from: /scratch/groups/jyeatman/samjohns-projects/unet3d/data/osfstorage-archive-test/N_435_final_crop_ds2.h5...
2023-05-17 00:12:43,115 [MainThread] INFO Dataset - Slice builder config: {'name': 'SliceBuilder', 'patch_shape': [80, 170, 170], 'stride_shape': [40, 90, 90]}
2023-05-17 00:12:

In [34]:
l = next(iter(loaders))  # if this is new to you, it's actually the *standard* way to use a PyTorch `DataLoader` object.

2023-05-17 00:19:20,062 [MainThread] INFO Dataset - Creating test set loaders...
2023-05-17 00:19:20,063 [MainThread] WARNING Dataset - Cannot find dataset class in the config. Using default 'StandardHDF5Dataset'.
2023-05-17 00:19:20,064 [MainThread] INFO HDF5Dataset - Loading test set from: /scratch/groups/jyeatman/samjohns-projects/unet3d/data/osfstorage-archive-test/N_511_final_crop_ds2.h5...
2023-05-17 00:19:20,689 [MainThread] INFO Dataset - Slice builder config: {'name': 'SliceBuilder', 'patch_shape': [80, 170, 170], 'stride_shape': [40, 90, 90]}
2023-05-17 00:19:20,691 [MainThread] INFO HDF5Dataset - Number of patches: 432
2023-05-17 00:19:20,692 [MainThread] INFO HDF5Dataset - Loading test set from: /scratch/groups/jyeatman/samjohns-projects/unet3d/data/osfstorage-archive-test/N_435_final_crop_ds2.h5...
2023-05-17 00:19:22,678 [MainThread] INFO Dataset - Slice builder config: {'name': 'SliceBuilder', 'patch_shape': [80, 170, 170], 'stride_shape': [40, 90, 90]}
2023-05-17 00:19:

In [61]:
x, y = next(iter(l))

In [62]:
x = x[0]  # but why??

In [63]:
x.shape

torch.Size([1, 80, 170, 170])

In [68]:
y[0][0]

slice(0, 80, None)

In [70]:
# Create the model
model = u3.model.get_model(config['model'])

In [71]:
model

UNet3D(
  (encoders): ModuleList(
    (0): Encoder(
      (basic_module): DoubleConv(
        (SingleConv1): SingleConv(
          (groupnorm): GroupNorm(1, 1, eps=1e-05, affine=True)
          (conv): Conv3d(1, 16, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
          (ReLU): ReLU(inplace=True)
        )
        (SingleConv2): SingleConv(
          (groupnorm): GroupNorm(8, 16, eps=1e-05, affine=True)
          (conv): Conv3d(16, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
          (ReLU): ReLU(inplace=True)
        )
      )
    )
    (1): Encoder(
      (pooling): MaxPool3d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (basic_module): DoubleConv(
        (SingleConv1): SingleConv(
          (groupnorm): GroupNorm(8, 32, eps=1e-05, affine=True)
          (conv): Conv3d(32, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
          (ReLU): ReLU(inplace=True)
        )
     

In [1]:
summary(model, ((1, 256, 256, 256)))

NameError: name 'summary' is not defined